# version for bunch of pdfs

In [1]:
import camelot
import pandas
pandas.options.display.max_colwidth = None

In [2]:
input_file = 'test.pdf'
# table_areas = top_left x, y, bottom_right x, y // 0,0 is bottom left
# read all pages in bottom 600x600 square
tables_transactions = camelot.read_pdf(input_file, flavor='stream', table_regions=['0,600,600,0'], pages='1-end')
tables_for_balance = camelot.read_pdf(input_file, pages='1-end')

trans_begin = tables_for_balance[0].df # take 3 lines
begin_month_balance = ''.join(trans_begin.iat[1,0].split())
begin_month_balance[28:]

trans_end = tables_for_balance[-1].df # take 3 lines
end_month_balance = ''.join(trans_end.iat[-2,0].split())
end_month_balance[28:]

In [3]:
def remove_whitespace(x):
    if x is not None:
        x = x.split()
        x = ''.join(x)
    return x


def extract_transactions(tables_transactions):
    """
    get the list of all monthly transactions, clean the data
    """
    transactions_list = []
    for table in tables_transactions:
        trans = table.df.values.tolist()
        for row in trans:
            transactions_list.append(row)
    transactions = pandas.DataFrame(transactions_list)

    transactions = transactions.drop(columns=[2,4,5])

    transactions = transactions.applymap(remove_whitespace)

    transactions.columns = ['info', 'date', 'out', 'in']

    transactions = transactions[transactions.date != 'Valuta']
    transactions = transactions[transactions.date != 'zu']
    transactions = transactions.fillna(value="")
    transactions.index = range(len(transactions.index))
    
    return transactions

In [4]:
def merge_transactions_3rows(transactions):
    """
    df -> list of lists
    merges 3 rows of one transaction into one, discard the rest
    """
    list_merged_transactions = []
    
    # month begin info
    start_info = ''
    month = ''
    balance = ''
    list_merged_transactions.append([start_info, month, '', balance])

    use_this_row = 0 # we'll use 3 consecutive rows, and merge them 
    for index, row in transactions.iterrows():
            if (row['date'] != ''):
                use_this_row = 1
                current_info = row['info']
                current_date = row['date']
                current_in = row['in']
                current_out = row['out']
            elif use_this_row > 0:
                use_this_row += 1
                if use_this_row <= 3: # unused row after row with date
                    current_info += ". . . ." + row['info']
                    if use_this_row == 3:
                        list_merged_transactions.append([current_info, current_date, current_out, current_in])
                        use_this_row = 0
    
    # add month end info
    return list_merged_transactions


In [5]:
final_transactions = pandas.DataFrame(merge_transactions_3rows(extract_transactions(tables_transactions)))
final_transactions.columns = ['info', 'date', 'out', 'in']

final_transactions[:5]